# Klimawandel-Erkennung in Kommentaren mit Qwen2.5-7B-Instruct-bnb-4bit

Dieses Notebook zeigt, wie man mithilfe des Modells **unsloth/Qwen2.5-7B-Instruct-bnb-4bit** jeden Kommentar aus einer zuvor gescrapten JSON‑Datei analysiert und prüft, ob der Begriff "Klimawandel" vorkommt.

Das Modell wird mittels einer Text-Generation-Pipeline eingesetzt und mit einem Prompt angewiesen, ausschließlich ein JSON-Objekt zurückzuliefern, das unter dem Schlüssel `klimawandel` einen Boolean-Wert (true/false) enthält.

**Wichtig:** Zunächst muss die Datei `taz_scraped_data_demo_2025-04-14.json` in das Arbeitsverzeichnis hochgeladen werden.

## 1. JSON-Datei Upload

Lade die Datei `taz_scraped_data_demo_2025-04-14.json` in dein Colab‑Arbeitsverzeichnis hoch.

In [ ]:
# Importiere benötigte Bibliotheken
import json
import os
from transformers import pipeline

print('Bibliotheken wurden erfolgreich importiert.')

In [ ]:
# Lade die JSON-Daten
json_filename = 'taz_scraped_data_demo_2025-04-14.json'
if not os.path.exists(json_filename):
    raise FileNotFoundError(f"Die Datei {json_filename} wurde nicht gefunden. Bitte lade sie hoch.")

with open(json_filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f'JSON-Datei "{json_filename}" erfolgreich geladen. Anzahl Artikel: {len(data)}')

## 2. Initialisierung der Text-Generation-Pipeline mit Qwen2.5-7B-Instruct-bnb-4bit

Wir verwenden die `text-generation`-Pipeline von Hugging Face, um das Modell **unsloth/Qwen2.5-7B-Instruct-bnb-4bit** zu laden. Dieses Modell benötigt keinen extra HF‑Token, sodass keine weiteren Einstellungen nötig sind.

Wir konfigurieren die Pipeline mit einer moderaten Ausgabe-Länge und deaktivieren das Sampling, um deterministische Ergebnisse zu erhalten.

In [ ]:
model_name = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"

generation_pipe = pipeline(
    "text-generation",
    model=model_name,
    tokenizer=model_name,
    max_new_tokens=60,
    do_sample=False
)

print(f'Text-Generation-Pipeline für {model_name} initialisiert.')

## 3. Funktion zur Klassifikation von Kommentaren

Die Funktion `classify_klimawandel` nimmt einen Kommentartext entgegen und übergibt diesen zusammen mit einem Prompt an das Modell. 

Der Prompt lautet beispielsweise:
```
Analysiere den folgenden Kommentartext und gib ausschließlich ein JSON-Objekt zurück, in dem der Schlüssel "klimawandel" mit true oder false belegt wird.
Kommentar: <Kommentartext>
```

Das Modell soll also z. B. folgendes zurückliefern:
```json
{ "klimawandel": true }
```

In [ ]:
def classify_klimawandel(comment_text):
    prompt = (
        'Analysiere den folgenden Kommentartext und gib ausschließlich ein JSON-Objekt zurück, '\n'
        'in dem der Schlüssel "klimawandel" mit true oder false belegt wird.\n' '\n'
        f'Kommentar: {comment_text}\n'
    )
    
    # Generiere eine Antwort mit dem Modell
    outputs = generation_pipe(prompt, clean_up_tokenization_spaces=True)
    response_text = outputs[0]['generated_text']
    
    # Versuche, ein JSON-Objekt aus der Ausgabe zu extrahieren
    try:
        json_start = response_text.find('{')
        json_str = response_text[json_start:]
        result = json.loads(json_str)
    except Exception as e:
        result = {"klimawandel": None, "error": str(e)}
    return result

# Teste die Funktion mit einem Beispielkommentar
example_comment = "Ich bin fest davon überzeugt, dass der Klimawandel aktuell eine große Herausforderung darstellt."
print('Test Output:', classify_klimawandel(example_comment))

## 4. Analyse der Kommentare

Nun iterieren wir über alle Artikel und deren Kommentare aus der JSON-Datei. Für jeden Kommentar wird das Modell aufgerufen, um zu prüfen, ob der Begriff "Klimawandel" vorkommt – die Ausgabe (als JSON) wird ausgegeben.

In [ ]:
for article_idx, article in enumerate(data):
    print(f"\n--- Artikel {article_idx+1} ---")
    comments = article.get('Comments', [])
    if not comments:
        print('Keine Kommentare vorhanden.')
    else:
        for com_idx, comment in enumerate(comments):
            comment_text = comment.get('Comment text', '')
            if not comment_text.strip():
                print(f"Kommentar {com_idx+1}: Kein Text vorhanden.")
            else:
                result = classify_klimawandel(comment_text)
                print(f"Kommentar {com_idx+1}: {result}")

print('\nAnalyse der Kommentare abgeschlossen.')

## Fazit

In diesem Notebook haben wir gezeigt, wie man mit dem Modell **unsloth/Qwen2.5-7B-Instruct-bnb-4bit** automatisch prüfen kann, ob in den Kommentaren der Begriff "Klimawandel" vorkommt. 

Die Methode demonstriert, wie man einen Prompt so gestaltet, dass ausschließlich ein strukturiertes JSON‑Objekt zurückgeliefert wird – ideal für automatisierte Auswertungen im Rahmen eines Kurses.